# CAFQA

In [1]:
import sys
sys.path.append("../")
from clapton.clapton import claptonize
from clapton.ansatzes import circular_ansatz,circular_ansatz_mirrored
from clapton.evaluation import get_energy
import numpy as np
from numbers import Number

In [2]:
def ising_model(N, Jx, h, Jy=0., periodic=False):
    """
    Constructs qubit Hamiltonian for linear Ising model.
    H = sum_{i=0...N-2} (Jx_i X_i X_{i+1} + Jy_i Y_i Y_{i+1}) + sum_{i=0...N-1}  h_i Z_i

    N (Int): # sites/qubits.
    Jx (Float, Iterable[Float]): XX strength, either constant value or list (values for each pair of neighboring sites).
    h (Float, Iterable[Float]): Z self-energy, either constant value or list (values for each site).
    Jy (Float, Iterable[Float]): YY strength, either constant value or list (values for each pair of neighboring sites).
    periodic: If periodic boundary conditions. If True, include term X_0 X_{N-1} and Y_0 Y_{N-1}.

    Returns:
    (Iterable[Float], Iterable[String], String) (Pauli coefficients, Pauli strings, "0"*N)
    """
    if isinstance(Jx, Number):
        if periodic:
            Jx = [Jx] * N
        else:
            Jx = [Jx] * (N-1)
    if isinstance(Jy, Number):
        if periodic:
            Jy = [Jy] * N
        else:
            Jy = [Jy] * (N-1)
    if isinstance(h, Number):
        h = [h] * N        
    if N > 1:
        assert len(Jx) == N if periodic else len(Jx) == N-1, "Jx has wrong length"
        assert len(Jy) == N if periodic else len(Jy) == N-1, "Jy has wrong length"
        assert len(h) == N, "h has wrong length"
    coeffs = []
    paulis = []
    # add XX terms
    for j in range(N-1):
        if np.abs(Jx[j]) > 1e-12:
            coeffs.append(Jx[j])
            paulis.append("I"*j+"XX"+"I"*(N-j-2))
    if N > 2 and periodic and np.abs(Jx[N-1]) > 1e-12:
        coeffs.append(Jx[N-1])
        paulis.append("X"+"I"*(N-2)+"X")
    # add YY terms
    for j in range(N-1):
        if np.abs(Jy[j]) > 1e-12:
            coeffs.append(Jy[j])
            paulis.append("I"*j+"YY"+"I"*(N-j-2))
    if N > 2 and periodic and np.abs(Jy[N-1]) > 1e-12:
        coeffs.append(Jy[N-1])
        paulis.append("Y"+"I"*(N-2)+"Y")
    # add Z terms
    for j in range(N):
        if np.abs(h[j]) > 1e-12:
            coeffs.append(h[j])
            paulis.append("I"*j+"Z"+"I"*(N-j-1))
    return coeffs, paulis, "1"*N

In [3]:
# set seed for reproducibility
np.random.seed(42)

# # define Hamiltonian, e.g. 3q Heisenberg model with random coefficients
# paulis = ["XXI", "IXX", "YYI", "IYY", "ZZI", "IZZ"]
# coeffs = np.random.random(len(paulis))
# paulis, coeffs

# 10 Qubit Hamiltonian
coeffs,paulis,_ = ising_model(N=10,Jx=0.2,Jy=0.3,h=0.4)



In [4]:
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit_algorithms import NumPyMinimumEigensolver

weights  =  coeffs
pauli_op = [([pauli,weight]) for pauli,weight in zip(paulis,weights)]
hamiltonian = SparsePauliOp.from_list([ op for op in pauli_op ])

numpy_solver = NumPyMinimumEigensolver()
result = numpy_solver.compute_minimum_eigenvalue(operator=hamiltonian)
ref_value = result.eigenvalue.real
print(f"Reference value: {ref_value:.5f}")

Reference value: -4.27192


In [5]:
# define parametrized Clifford circuit that is being optimized over
# here we use the circular_ansatz template
# we fix 2q gates as they will not be optimized over
vqe_pcirc = circular_ansatz(N=len(paulis[0]), reps=10, fix_2q=True)

In [6]:
# the circuit consists of parametrized gates
for gate in vqe_pcirc.gates:
    print(gate.label, gate.is_fixed())

RY False
RY False
RY False
RY False
RY False
RY False
RY False
RY False
RY False
RY False
RZ False
RZ False
RZ False
RZ False
RZ False
RZ False
RZ False
RZ False
RZ False
RZ False
2Q True
2Q True
2Q True
2Q True
2Q True
2Q True
2Q True
2Q True
2Q True
2Q True
RY False
RY False
RY False
RY False
RY False
RY False
RY False
RY False
RY False
RY False
RZ False
RZ False
RZ False
RZ False
RZ False
RZ False
RZ False
RZ False
RZ False
RZ False
2Q True
2Q True
2Q True
2Q True
2Q True
2Q True
2Q True
2Q True
2Q True
2Q True
RY False
RY False
RY False
RY False
RY False
RY False
RY False
RY False
RY False
RY False
RZ False
RZ False
RZ False
RZ False
RZ False
RZ False
RZ False
RZ False
RZ False
RZ False
2Q True
2Q True
2Q True
2Q True
2Q True
2Q True
2Q True
2Q True
2Q True
2Q True
RY False
RY False
RY False
RY False
RY False
RY False
RY False
RY False
RY False
RY False
RZ False
RZ False
RZ False
RZ False
RZ False
RZ False
RZ False
RZ False
RZ False
RZ False
2Q True
2Q True
2Q True
2Q True
2Q True


In [7]:
# non-fixed gates will be optimized over
# RY and RZ gates can assume 4 values k = 0,1,2,3 which describe multiples of pi/2

In [8]:
# the initial parameters are all 0
vqe_pcirc.read()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [9]:
# we can look at the corresponding stim circuit
vqe_pcirc.stim_circuit().diagram()

q0: -I-I-X-@---------------I-I---X-@---------------I-I---X-@---------------I-I---X-@---------------I-I---X-@---------------I-I---X-@---------------I-I---X-@---------------I-I---X-@---------------I-I---X-@---------------I-I---X-@---------------I-I---
         | |                     | |                     | |                     | |                     | |                     | |                     | |                     | |                     | |                     | |
q1: -I-I-|-X-@-------------I-I---|-X-@-------------I-I---|-X-@-------------I-I---|-X-@-------------I-I---|-X-@-------------I-I---|-X-@-------------I-I---|-X-@-------------I-I---|-X-@-------------I-I---|-X-@-------------I-I---|-X-@-------------I-I---
         |   |                   |   |                   |   |                   |   |                   |   |                   |   |                   |   |                   |   |                   |   |                   |   |
q2: -I-I-|---X-@-----------I-I---|---X-@-----------I-I---|---X-@-----------I-I---|---X-@-----------I-I---|---X-@-----------I-I---|---X-@-----------I-I---|---X-@-----------I-I---|---X-@-----------I-I---|---X-@-----------I-I---|---X-@-----------I-I---
         |     |                 |     |                 |     |                 |     |                 |     |                 |     |                 |     |                 |     |                 |     |                 |     |
q3: -I-I-|-----X-@---------I-I---|-----X-@---------I-I---|-----X-@---------I-I---|-----X-@---------I-I---|-----X-@---------I-I---|-----X-@---------I-I---|-----X-@---------I-I---|-----X-@---------I-I---|-----X-@---------I-I---|-----X-@---------I-I---
         |       |               |       |               |       |               |       |               |       |               |       |               |       |               |       |               |       |               |       |
q4: -I-I-|-------X-@-------I-I---|-------X-@-------I-I---|-------X-@-------I-I---|-------X-@-------I-I---|-------X-@-------I-I---|-------X-@-------I-I---|-------X-@-------I-I---|-------X-@-------I-I---|-------X-@-------I-I---|-------X-@-------I-I---
         |         |             |         |             |         |             |         |             |         |             |         |             |         |             |         |             |         |             |         |
q5: -I-I-|---------X-@-----I-I---|---------X-@-----I-I---|---------X-@-----I-I---|---------X-@-----I-I---|---------X-@-----I-I---|---------X-@-----I-I---|---------X-@-----I-I---|---------X-@-----I-I---|---------X-@-----I-I---|---------X-@-----I-I---
         |           |           |           |           |           |           |           |           |           |           |           |           |           |           |           |           |           |           |           |
q6: -I-I-|-----------X-@---I-I---|-----------X-@---I-I---|-----------X-@---I-I---|-----------X-@---I-I---|-----------X-@---I-I---|-----------X-@---I-I---|-----------X-@---I-I---|-----------X-@---I-I---|-----------X-@---I-I---|-----------X-@---I-I---
         |             |         |             |         |             |         |             |         |             |         |             |         |             |         |             |         |             |         |             |
q7: -I-I-|-------------X-@-I-I---|-------------X-@-I-I---|-------------X-@-I-I---|-------------X-@-I-I---|-------------X-@-I-I---|-------------X-@-I-I---|-------------X-@-I-I---|-------------X-@-I-I---|-------------X-@-I-I---|-------------X-@-I-I---
         |               |       |               |       |               |       |               |       |               |       |               |       |               |       |               |       |               |       |               |
q8: -I-I-|---------------X-@-I-I-|---------------X-@-I-I-|---------------X-@-I-I-|---------------X-@-I-I-|------

In [10]:
# we can assign a different set of parameters
# vqe_pcirc.assign([0,1,2,3,0,1,2,3,0,1,2,3])

In [11]:
vqe_pcirc.stim_circuit().diagram()

q0: -I-I-X-@---------------I-I---X-@---------------I-I---X-@---------------I-I---X-@---------------I-I---X-@---------------I-I---X-@---------------I-I---X-@---------------I-I---X-@---------------I-I---X-@---------------I-I---X-@---------------I-I---
         | |                     | |                     | |                     | |                     | |                     | |                     | |                     | |                     | |                     | |
q1: -I-I-|-X-@-------------I-I---|-X-@-------------I-I---|-X-@-------------I-I---|-X-@-------------I-I---|-X-@-------------I-I---|-X-@-------------I-I---|-X-@-------------I-I---|-X-@-------------I-I---|-X-@-------------I-I---|-X-@-------------I-I---
         |   |                   |   |                   |   |                   |   |                   |   |                   |   |                   |   |                   |   |                   |   |                   |   |
q2: -I-I-|---X-@-----------I-I---|---X-@-----------I-I---|---X-@-----------I-I---|---X-@-----------I-I---|---X-@-----------I-I---|---X-@-----------I-I---|---X-@-----------I-I---|---X-@-----------I-I---|---X-@-----------I-I---|---X-@-----------I-I---
         |     |                 |     |                 |     |                 |     |                 |     |                 |     |                 |     |                 |     |                 |     |                 |     |
q3: -I-I-|-----X-@---------I-I---|-----X-@---------I-I---|-----X-@---------I-I---|-----X-@---------I-I---|-----X-@---------I-I---|-----X-@---------I-I---|-----X-@---------I-I---|-----X-@---------I-I---|-----X-@---------I-I---|-----X-@---------I-I---
         |       |               |       |               |       |               |       |               |       |               |       |               |       |               |       |               |       |               |       |
q4: -I-I-|-------X-@-------I-I---|-------X-@-------I-I---|-------X-@-------I-I---|-------X-@-------I-I---|-------X-@-------I-I---|-------X-@-------I-I---|-------X-@-------I-I---|-------X-@-------I-I---|-------X-@-------I-I---|-------X-@-------I-I---
         |         |             |         |             |         |             |         |             |         |             |         |             |         |             |         |             |         |             |         |
q5: -I-I-|---------X-@-----I-I---|---------X-@-----I-I---|---------X-@-----I-I---|---------X-@-----I-I---|---------X-@-----I-I---|---------X-@-----I-I---|---------X-@-----I-I---|---------X-@-----I-I---|---------X-@-----I-I---|---------X-@-----I-I---
         |           |           |           |           |           |           |           |           |           |           |           |           |           |           |           |           |           |           |           |
q6: -I-I-|-----------X-@---I-I---|-----------X-@---I-I---|-----------X-@---I-I---|-----------X-@---I-I---|-----------X-@---I-I---|-----------X-@---I-I---|-----------X-@---I-I---|-----------X-@---I-I---|-----------X-@---I-I---|-----------X-@---I-I---
         |             |         |             |         |             |         |             |         |             |         |             |         |             |         |             |         |             |         |             |
q7: -I-I-|-------------X-@-I-I---|-------------X-@-I-I---|-------------X-@-I-I---|-------------X-@-I-I---|-------------X-@-I-I---|-------------X-@-I-I---|-------------X-@-I-I---|-------------X-@-I-I---|-------------X-@-I-I---|-------------X-@-I-I---
         |               |       |               |       |               |       |               |       |               |       |               |       |               |       |               |       |               |       |               |
q8: -I-I-|---------------X-@-I-I-|---------------X-@-I-I-|---------------X-@-I-I-|---------------X-@-I-I-|------

In [12]:
# we can perform CAFQA by using the main optimization function "claptonize"
ks_best, _, energy_best = claptonize(
    paulis,
    coeffs,
    vqe_pcirc,
    n_proc=4,           # total number of processes in parallel
    n_starts=4,         # number of random genetic algorithm starts in parallel
    n_rounds=1,         # number of budget rounds, if None it will terminate itself
    callback=print,     # callback for internal parameter (#iteration, energies, ks) processing
    budget=20           # budget per genetic algorithm instance
)

STARTING ROUND 0


started GA at id 1 with 1 procs

started GA at id 2 with 1 procs

started GA at id 3 with 1 procs



/home/dbharadwaj/anaconda3/envs/clapton/lib/python3.10/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")
/home/dbharadwaj/anaconda3/envs/clapton/lib/python3.10/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evo

started GA at id None with 1 procs



/home/dbharadwaj/anaconda3/envs/clapton/lib/python3.10/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")
/home/dbharadwaj/anaconda3/envs/clapton/lib/python3.10/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evo

[0, array([0., 0., 0., 0.]), array([3, 3, 0, 2, 3, 3, 2, 3, 2, 1, 1, 2, 1, 0, 2, 1, 2, 0, 0, 2, 3, 0,
       2, 3, 2, 1, 3, 3, 2, 0, 0, 0, 3, 0, 3, 2, 1, 2, 0, 1, 1, 1, 1, 3,
       0, 0, 2, 3, 0, 2, 2, 0, 2, 1, 2, 3, 0, 3, 2, 1, 2, 1, 1, 1, 0, 2,
       3, 0, 0, 1, 1, 0, 0, 3, 2, 1, 1, 3, 2, 3, 3, 2, 0, 2, 0, 3, 2, 1,
       1, 0, 2, 0, 1, 2, 1, 2, 3, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 3, 0, 2,
       0, 0, 0, 1, 1, 0, 3, 1, 0, 0, 3, 0, 2, 0, 1, 0, 2, 2, 3, 1, 0, 3,
       0, 0, 3, 1, 2, 2, 3, 1, 1, 0, 1, 1, 2, 2, 0, 3, 1, 0, 3, 3, 2, 2,
       3, 2, 1, 0, 3, 0, 2, 0, 2, 1, 1, 3, 2, 2, 2, 1, 2, 3, 3, 0, 0, 1,
       2, 1, 1, 1, 3, 3, 3, 0, 3, 3, 0, 1, 3, 0, 2, 1, 3, 3, 0, 0, 3, 2,
       2, 3, 0, 3, 1, 0, 1, 0, 3, 3, 2, 0, 1, 0, 0, 0, 1, 0, 1, 2, 2, 1],
      dtype=object)]
[0, array([0., 0., 0., 0.]), array([3, 3, 0, 2, 3, 3, 2, 3, 2, 1, 1, 2, 1, 0, 2, 1, 2, 0, 0, 2, 3, 0,
       2, 3, 2, 1, 3, 3, 2, 0, 0, 0, 3, 0, 3, 2, 1, 2, 0, 1, 1, 1, 1, 3,
       0, 0, 2, 3, 0, 2, 2, 0, 2, 1, 2, 3, 0

In [13]:
# the best parameters are
ks_best

[2,
 0,
 1,
 1,
 3,
 0,
 1,
 0,
 2,
 0,
 2,
 0,
 2,
 3,
 1,
 0,
 2,
 3,
 3,
 2,
 1,
 3,
 2,
 1,
 2,
 0,
 0,
 3,
 0,
 1,
 0,
 3,
 0,
 0,
 2,
 2,
 0,
 2,
 3,
 1,
 2,
 2,
 3,
 3,
 3,
 3,
 0,
 1,
 3,
 0,
 0,
 3,
 1,
 2,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 1,
 1,
 3,
 3,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 3,
 3,
 0,
 3,
 0,
 2,
 3,
 1,
 3,
 0,
 3,
 3,
 0,
 3,
 3,
 0,
 3,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 1,
 2,
 2,
 0,
 3,
 0,
 3,
 1,
 0,
 0,
 2,
 1,
 3,
 3,
 0,
 0,
 2,
 0,
 0,
 3,
 2,
 2,
 0,
 0,
 1,
 0,
 1,
 0,
 3,
 1,
 1,
 3,
 3,
 2,
 3,
 1,
 2,
 3,
 0,
 0,
 3,
 2,
 0,
 3,
 3,
 2,
 3,
 0,
 1,
 0,
 0,
 1,
 1,
 3,
 0,
 1,
 0,
 3,
 1,
 2,
 2,
 0,
 2,
 1,
 3,
 0,
 2,
 0,
 1,
 3,
 2,
 0,
 3,
 3,
 2,
 0,
 2,
 3,
 1,
 1,
 0,
 3,
 2,
 0,
 1,
 1,
 0,
 0,
 1,
 2,
 1,
 0,
 2,
 2,
 3,
 2,
 0,
 1,
 0,
 3,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 2,
 3,
 2,
 1,
 0,
 3,
 1,
 2,
 0,
 3,
 2,
 0,
 1]

In [14]:
# with energy
energy_best

0.0

In [15]:
# the corresponding circuit is
vqe_pcirc.assign(ks_best)
vqe_pcirc.stim_circuit().diagram()

q0: -Y----------Z-----X-@---------------SQRT_Y-----I------------X-@---------------Y----------I------------X-@---------------I----------I-----------X-@---------------Y----------I------------X-@---------------SQRT_Y-----Z-----------X-@---------------Y------S------------X-@---------------SQRT_Y_DAG-I--------X-@---------------Y----------Z------------X-@---------------I----------S------------X-@---------------I------S------------
                      | |                                       | |                                       | |                                      | |                                       | |                                      | |                                   | |                                   | |                                       | |                                       | |
q1: -I----------I-----|-X-@-------------SQRT_Y_DAG-S_DAG--------|-X-@-------------Y----------S_DAG--------|-X-@-------------Y----------I-----------|-X-@-------------SQRT_Y_DAG-S_DAG--------|-X-@-------------Y----------S-----------|-X-@-------------Y------S_DAG--------|-X-@-------------Y----------S--------|-X-@-------------I----------I------------|-X-@-------------SQRT_Y_DAG-I------------|-X-@-------------I------I------------
                      |   |                                     |   |                                     |   |                                    |   |                                     |   |                                    |   |                                 |   |                                 |   |                                     |   |                                     |   |
q2: -SQRT_Y-----Z-----|---X-@-----------Y----------I------------|---X-@-----------SQRT_Y_DAG-S------------|---X-@-----------I----------I-----------|---X-@-----------SQRT_Y-----Z------------|---X-@-----------Y----------S_DAG-------|---X-@-----------I------S_DAG--------|---X-@-----------I----------S--------|---X-@-----------Y----------S_DAG--------|---X-@-----------Y----------Z------------|---X-@-----------I------S_DAG--------
                      |     |                                   |     |                                   |     |                                  |     |                                   |     |                                  |     |                               |     |                               |     |                                   |     |                                   |     |
q3: -SQRT_Y-----S_DAG-|-----X-@---------SQRT_Y-----I------------|-----X-@---------SQRT_Y_DAG-Z------------|-----X-@---------SQRT_Y-----S-----------|-----X-@---------SQRT_Y_DAG-Z------------|-----X-@---------I----------S_DAG-------|-----X-@---------I------Z------------|-----X-@---------SQRT_Y_DAG-S_DAG----|-----X-@---------SQRT_Y-----S_DAG--------|-----X-@---------I----------Z------------|-----X-@---------Y------S------------
                      |       |                                 |       |                                 |       |                                |       |                                 |       |                                |       |                             |       |                             |       |                                 |       |                                 |       |
q4: -SQRT_Y_DAG-S-----|-------X-@-------Y----------Z------------|-------X-@-------SQRT_Y_DAG-Z------------|-------X-@-------SQRT_Y-----I-----------|-------X-@-------I----------Z------------|-------X-@-------SQRT_Y_DAG-I-----------|-------X-@-------SQRT_Y-S_DAG--------|-------X-@-------SQRT_Y_DAG-I--------|-------X-@-------SQRT_Y_DAG-Z------------|-------X-@-------SQRT_Y-----S_DAG--------|-------X-@-------I------Z------------
                      |         |                               |         |                               |         |                              |         |                               |         |                              |         |                          

# noisy CAFQA (nCAFQA)

In [16]:
from clapton.depolarization import GateGeneralDepolarizationModel

In [17]:
# let's add a noise model where we specify global 1q and 2q gate errors
nm = GateGeneralDepolarizationModel(p1=0.005, p2=0.05) #NOTE: This is the Noise Model, not representative of device noise

In [18]:
vqe_pcirc = circular_ansatz(N=len(paulis[0]), reps=10, fix_2q=True)
vqe_pcirc.add_depolarization_model(nm)

In [19]:
# after every gate a depol channel is added
vqe_pcirc.stim_circuit().diagram()

q0: -I-DEPOLARIZE1(0.005)---------------------------------------------------------------------------------------------------------------------------------------------------------I------------------DEPOLARIZE1(0.005)----------------------------------------------------------------------------------------------------------------------------------------------------------------------------X-DEPOLARIZE2(0.05)-@-DEPOLARIZE2(0.05)-----------------------------------------------------------------------------------------------------------------------------------------------I-----------------DEPOLARIZE1(0.005)---------------------------------------------------------------------------------------------------------------------------------------------------------I------------------DEPOLARIZE1(0.005)----------------------------------------------------------------------------------------------------------------------------------------------------------------------------X-DEPOLARIZE2(0.05)-@-DEPOLARIZE2(0.05)-----------------------------------------------------------------------------------------------------------------------------------------------I-----------------DEPOLARIZE1(0.005)---------------------------------------------------------------------------------------------------------------------------------------------------------I------------------DEPOLARIZE1(0.005)----------------------------------------------------------------------------------------------------------------------------------------------------------------------------X-DEPOLARIZE2(0.05)-@-DEPOLARIZE2(0.05)-----------------------------------------------------------------------------------------------------------------------------------------------I-----------------DEPOLARIZE1(0.005)---------------------------------------------------------------------------------------------------------------------------------------------------------I------------------DEPOLARIZE1(0.005)----------------------------------------------------------------------------------------------------------------------------------------------------------------------------X-DEPOLARIZE2(0.05)-@-DEPOLARIZE2(0.05)-----------------------------------------------------------------------------------------------------------------------------------------------I-----------------DEPOLARIZE1(0.005)---------------------------------------------------------------------------------------------------------------------------------------------------------I------------------DEPOLARIZE1(0.005)----------------------------------------------------------------------------------------------------------------------------------------------------------------------------X-DEPOLARIZE2(0.05)-@-DEPOLARIZE2(0.05)-----------------------------------------------------------------------------------------------------------------------------------------------I-----------------DEPOLARIZE1(0.005)---------------------------------------------------------------------------------------------------------------------------------------------------------I------------------DEPOLARIZE1(0.005)----------------------------------------------------------------------------------------------------------------------------------------------------------------------------X-DEPOLARIZE2(0.05)-@-DEPOLARIZE2(0.05)-----------------------------------------------------------------------------------------------------------------------------------------------I-----------------DEPOLARIZE1(0.005)---------------------------------------------------------------------------------------------------------------------------------------------------------I------------------DEPOLARIZE1(0.005)----------------------------------------------------------------------------------------------------------------------------------------------------------------------------X-DEPOLARIZE2(0.05)-@-DEPOLARIZE2(0.05)----------------------------------------------------------------------------------------------

In [ ]:
# we can perform nCAFQA by using the main optimization function "claptonize"
# now with the noisy circuit
ks_best, energy_noisy, energy_noiseless = claptonize(
    paulis,
    coeffs,
    vqe_pcirc,
    n_proc=4,           # total number of processes in parallel
    n_starts=4,         # number of random genetic algorithm starts in parallel
    n_rounds=1,         # number of budget rounds, if None it will terminate itself
    callback=print,     # callback for internal parameter (#iteration, energies, ks) processing
    budget=20           # budget per genetic algorithm instance
)

STARTING ROUND 0


started GA at id 1 with 1 procs

started GA at id 2 with 1 procs


started GA at id 3 with 1 procs
started GA at id None with 1 procs

[0, array([-0.02998, -0.02998,  0.     ,  0.     ]), array([3, 1, 2, 2, 2, 2, 1, 1, 1, 3, 0, 0, 3, 3, 2, 0, 1, 0, 2, 1, 3, 1,
       3, 3, 0, 2, 2, 0, 3, 1, 1, 1, 1, 3, 2, 2, 1, 1, 1, 0, 3, 2, 2, 2,
       1, 3, 0, 3, 2, 1, 0, 3, 3, 3, 0, 1, 3, 0, 3, 0, 0, 0, 1, 0, 2, 1,
       2, 0, 3, 0, 1, 2, 3, 3, 1, 2, 1, 3, 2, 1, 3, 1, 0, 2, 1, 1, 3, 3,
       1, 1, 3, 2, 1, 1, 1, 3, 3, 3, 1, 2, 2, 3, 0, 0, 1, 2, 3, 0, 2, 3,
       2, 2, 0, 2, 0, 2, 1, 2, 2, 3, 3, 3, 0, 0, 1, 3, 0, 1, 3, 0, 0, 3,
       0, 2, 1, 3, 3, 2, 0, 1, 0, 0, 2, 3, 2, 0, 2, 2, 2, 2, 1, 3, 3, 3,
       0, 2, 2, 3, 2, 1, 3, 3, 2, 0, 0, 3, 3, 1, 2, 3, 2, 2, 2, 0, 0, 0,
       2, 0, 1, 1, 1, 2, 2, 2, 3, 2, 1, 2, 0, 0, 1, 0, 2, 1, 3, 2, 0, 0,
       3, 1, 2, 0, 1, 3, 2, 1, 2, 0, 1, 0, 0, 0, 0, 3, 3, 3, 3, 3, 0, 3],
      dtype=object)]
[0, array([-0.04214, -0.04214,  0.     , 

In [21]:
# the best parameters are
ks_best

[0,
 0,
 1,
 3,
 3,
 0,
 3,
 1,
 3,
 3,
 1,
 3,
 2,
 0,
 1,
 1,
 0,
 1,
 0,
 2,
 0,
 0,
 2,
 0,
 3,
 0,
 1,
 3,
 1,
 3,
 0,
 3,
 1,
 0,
 3,
 2,
 2,
 3,
 0,
 3,
 0,
 0,
 3,
 0,
 0,
 2,
 0,
 1,
 3,
 0,
 3,
 2,
 0,
 2,
 2,
 1,
 3,
 0,
 0,
 3,
 1,
 3,
 2,
 2,
 3,
 0]

In [22]:
# with noisy/noiseless energy
energy_noisy, energy_noiseless

(0.0932051265125863, 0.5986584841970366)

In [23]:
# differrence
np.abs(energy_noisy-energy_noiseless)

0.5054533576844503

In [24]:
# the corresponding circuit is
vqe_pcirc.assign(ks_best)
vqe_pcirc.snapshot_noiseless().circ_snapshot_noiseless.diagram()

q0: -I------S_DAG-X-@-SQRT_Y_DAG-S_DAG------------X-@-Y-S--------X-@-I-I---X-@-SQRT_Y_DAG-S_DAG--------X-@-I-I----------------X-@-Y-S_DAG--------X-@-SQRT_Y_DAG-Z---X-@-SQRT_Y_DAG-Z------------X-@-Y-I----------------X-@-SQRT_Y-Z----------------
                  | |                             | |            | |       | |                         | |                    | |                | |                | |                         | |                    | |
q1: -I------S_DAG-|-X-@----------SQRT_Y-----S-----|-X-@-I------I-|-X-@-Y-Z-|-X-@----------I------S-----|-X-@-SQRT_Y_DAG-S_DAG-|-X-@-SQRT_Y_DAG-I-|-X-@----------I-I-|-X-@----------I----------I-|-X-@-SQRT_Y-----I-----|-X-@------SQRT_Y_DAG-S_DAG-
                  |   |                           |   |          |   |     |   |                       |   |                  |   |              |   |              |   |                       |   |                  |   |
q2: -SQRT_Y-I-----@---X----------SQRT_Y_DAG-S_DAG-@---X-SQRT_Y-S-@---X-I-I-@---X----------SQRT_Y-S_DAG-@---X-SQRT_Y-----Z-----@---X-I----------I-@---X----------I-S-@---X----------SQRT_Y_DAG-Z-@---X-SQRT_Y_DAG-S_DAG-@---X------Y----------I-----

## Comparing Initial Energy States

## Let's do one VQE iteration now